In [1]:
import pandas as pd
import numpy as np
import os
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
matplotlib.rcParams['axes.unicode_minus'] = False
pd.options.mode.chained_assignment = None  # default='warn'
plt.rcParams['font.family'] = 'Malgun Gothic'
import datetime
import time

In [2]:
def day_modifier(x):
    # x is a number
    
    if pd.isna(x) == True:
        return x
    else:
        x = str(x)
        if len(x) == 1:
            return '0' + x
        else:
            return x

In [3]:
def landnum_modifier(x):
    # x is a string
    
    if pd.isna(x) == True:
        return x
    else:
        x = str(x).replace('외', '').replace(' ','')
        splitted = x.split('-')
        if len(splitted) == 1:
            return x + '-0'
        else:
            return x

In [4]:
def data_prep(bdtype, tradetype):
    # bdtype is a string: one of 아파트, 연립다세대 or 오피스텔
    # tradetype is a string: one of 매매 or 전월세
    
    basedir = './국토교통부_실거래가_공개시스템/{}/{}/'.format(bdtype, tradetype)
    filenames = [f for f in os.listdir(basedir) if (f.endswith('.csv'))&('(' in f)]
    
    dfs_list = []
    for i, f in tqdm(enumerate(filenames)):
        df = pd.read_csv(basedir + f, encoding='euc-kr', header=15)
        if '해제사유발생일' in df.columns.tolist():
            df = df[df['해제사유발생일'].isna()]
            df = df.drop(columns=['해제사유발생일'])

        dfs_list.append(df)
    
    concat_df = pd.concat(dfs_list).reset_index(drop=True)
        
    concat_df['번지'] = concat_df['번지'].apply(landnum_modifier)
    
    concat_df['계약년월'] = concat_df['계약년월'].apply(str)
    concat_df['계약일'] = concat_df['계약일'].apply(str)
    
    concat_df['계약날짜기준_건물연식'] = concat_df['계약년월'].apply(lambda x: int(x[:4])) - concat_df['건축년도']
    
    concat_df['계약일'] = concat_df['계약일'].apply(day_modifier)
    
    concat_df['계약날짜'] = concat_df['계약년월'].apply(lambda x: x[:4]) + '-' + concat_df['계약년월'].apply(lambda x: x[-2:])\
    + '-' + concat_df['계약일']
    
    concat_df['계약날짜'] = pd.to_datetime(concat_df['계약날짜'], format='%Y-%m-%d')
    
    concat_df['거래금액(만원)'] = concat_df['거래금액(만원)'].apply(lambda x: int(x.replace(',','')))
    concat_df['단가(만원/㎡)'] = concat_df['거래금액(만원)'] / concat_df['전용면적(㎡)']
    
    concat_df['지번주소'] = concat_df['시군구'] + ' ' + concat_df['번지']
        
    cols_to_drop = ['시군구', '번지', '본번', '부번', '지번주소', '계약년월', '계약일', '도로명']
        
    concat_df = concat_df[['지번주소', '도로명'] + [col for col in concat_df.columns if col not in cols_to_drop]]
    
    date_today = pd.to_datetime(datetime.datetime.now().strftime('%Y-%m-%d'), format='%Y-%m-%d')
    concat_df['건물연식'] = date_today.year - concat_df['건축년도']
    
    concat_df = concat_df.dropna(subset=['지번주소'])
    
    if '건물명' in concat_df.columns:
        concat_df.rename(columns={'건물명':'건물(단지)명'}, inplace=True)
    elif '단지명' in concat_df.columns:
        concat_df.rename(columns={'단지명':'건물(단지)명'}, inplace=True)
    
    concat_df = concat_df[concat_df['층'] >= 0].reset_index(drop=True)
    
    return concat_df

In [5]:
yunrip_df = data_prep('연립다세대', '매매')
yunrip_df = yunrip_df[(yunrip_df['계약날짜'] >= pd.to_datetime('2015-01-01'))
                      &(yunrip_df['계약날짜기준_건물연식'] <= 5)
                     ]
yunrip_df = yunrip_df.reset_index(drop=True)
yunrip_df['부동산유형'] = '연립다세대'
print(yunrip_df.shape)
yunrip_df.head()

16it [00:01, 10.44it/s]


(101278, 13)


,지번주소,도로명,건물(단지)명,전용면적(㎡),대지권면적(㎡),거래금액(만원),층,건축년도,계약날짜기준_건물연식,계약날짜,단가(만원/㎡),건물연식,부동산유형
0,서울특별시 강남구 개포동 1216-4,개포로26길 16,(1216-4),33.77,21.67,24800,4,2012.0,3.0,2015-03-26,734.379627,9.0,연립다세대
1,서울특별시 강남구 개포동 1216-4,개포로26길 16,(1216-4),35.87,23.02,27200,4,2012.0,3.0,2015-06-23,758.293839,9.0,연립다세대
2,서울특별시 강남구 개포동 1216-4,개포로26길 16,(1216-4),29.12,18.68,22200,4,2012.0,3.0,2015-07-20,762.362637,9.0,연립다세대
3,서울특별시 강남구 개포동 1216-4,개포로26길 16,(1216-4),29.97,19.23,22500,3,2012.0,3.0,2015-08-06,750.750751,9.0,연립다세대
4,서울특별시 강남구 개포동 1169-5,논현로6길 10-14,RAON,47.19,28.71,46500,3,2015.0,0.0,2015-10-22,985.378258,6.0,연립다세대


In [6]:
yunrip_df.drop(columns=['대지권면적(㎡)'], inplace=True)
print(yunrip_df.shape)
yunrip_df.head()

(101278, 12)


,지번주소,도로명,건물(단지)명,전용면적(㎡),거래금액(만원),층,건축년도,계약날짜기준_건물연식,계약날짜,단가(만원/㎡),건물연식,부동산유형
0,서울특별시 강남구 개포동 1216-4,개포로26길 16,(1216-4),33.77,24800,4,2012.0,3.0,2015-03-26,734.379627,9.0,연립다세대
1,서울특별시 강남구 개포동 1216-4,개포로26길 16,(1216-4),35.87,27200,4,2012.0,3.0,2015-06-23,758.293839,9.0,연립다세대
2,서울특별시 강남구 개포동 1216-4,개포로26길 16,(1216-4),29.12,22200,4,2012.0,3.0,2015-07-20,762.362637,9.0,연립다세대
3,서울특별시 강남구 개포동 1216-4,개포로26길 16,(1216-4),29.97,22500,3,2012.0,3.0,2015-08-06,750.750751,9.0,연립다세대
4,서울특별시 강남구 개포동 1169-5,논현로6길 10-14,RAON,47.19,46500,3,2015.0,0.0,2015-10-22,985.378258,6.0,연립다세대


In [7]:
officetel_df = data_prep('오피스텔', '매매')
officetel_df = officetel_df[(officetel_df['계약날짜'] >= pd.to_datetime('2015-01-01'))
                            &(officetel_df['계약날짜기준_건물연식'] <= 5)
                           ]
officetel_df = officetel_df.reset_index(drop=True)
officetel_df['부동산유형'] = '오피스텔'
print(officetel_df.shape)
officetel_df.head()

16it [00:00, 40.10it/s]


(21860, 12)


,지번주소,도로명,건물(단지)명,전용면적(㎡),거래금액(만원),층,건축년도,계약날짜기준_건물연식,계약날짜,단가(만원/㎡),건물연식,부동산유형
0,서울특별시 강남구 삼성동 58-2,영동대로 647,청담역 푸르지오시티,29.32,32180,10,2014.0,1.0,2015-01-12,1097.544338,7.0,오피스텔
1,서울특별시 강남구 삼성동 58-2,영동대로 647,청담역 푸르지오시티,25.84,32900,19,2014.0,1.0,2015-01-14,1273.219814,7.0,오피스텔
2,서울특별시 강남구 삼성동 58-2,영동대로 647,청담역 푸르지오시티,25.84,38000,20,2014.0,1.0,2015-01-15,1470.588235,7.0,오피스텔
3,서울특별시 강남구 삼성동 58-2,영동대로 647,청담역 푸르지오시티,29.32,34000,5,2014.0,1.0,2015-02-24,1159.618008,7.0,오피스텔
4,서울특별시 강남구 삼성동 58-2,영동대로 647,청담역 푸르지오시티,25.84,32000,8,2014.0,1.0,2015-03-02,1238.390093,7.0,오피스텔


In [8]:
concat_df = pd.concat([yunrip_df, officetel_df]).sort_values(['지번주소', '건축년도', '계약날짜', '전용면적(㎡)']).reset_index(drop=True)
print(concat_df.shape)
concat_df.head()

(123138, 12)


,지번주소,도로명,건물(단지)명,전용면적(㎡),거래금액(만원),층,건축년도,계약날짜기준_건물연식,계약날짜,단가(만원/㎡),건물연식,부동산유형
0,서울특별시 강남구 개포동 1164-0,논현로2길 6,골드캐슬,58.05,57500,2,2017.0,1.0,2018-01-20,990.525409,4.0,연립다세대
1,서울특별시 강남구 개포동 1164-0,논현로2길 6,골드캐슬,58.05,57000,1,2017.0,1.0,2018-06-01,981.912145,4.0,연립다세대
2,서울특별시 강남구 개포동 1164-0,논현로2길 6,골드캐슬,58.05,63000,2,2017.0,2.0,2019-01-19,1085.271318,4.0,연립다세대
3,서울특별시 강남구 개포동 1164-26,논현로2길 64,슈베르트빌,57.67,52000,5,2015.0,0.0,2015-05-15,901.681984,6.0,연립다세대
4,서울특별시 강남구 개포동 1164-26,논현로2길 64,슈베르트빌,72.44,56000,5,2015.0,0.0,2015-06-17,773.053562,6.0,연립다세대


In [9]:
def find_bulktrade(df):
    idx_list = []
    for addr in tqdm(df['지번주소'].unique()):
        addr_df = df[df['지번주소'] == addr]
        
        for built_year in addr_df['건축년도'].unique():
            built_year_df = addr_df[addr_df['건축년도'] == built_year]

            for date in built_year_df['계약날짜'].unique():
                date_df = built_year_df[built_year_df['계약날짜'] == date]

                if date_df.shape[0] >= 10:
                    [idx_list.append(idx) for idx in date_df.index]
    
    selected_df = df.loc[idx_list,:]
    
    return selected_df

In [10]:
concat_selected_df = find_bulktrade(concat_df)
print(concat_selected_df.shape)
concat_selected_df.head()

100%|███████████████████████████████████████████████████████████████████████████| 17643/17643 [02:13<00:00, 132.64it/s]

(30157, 12)


,지번주소,도로명,건물(단지)명,전용면적(㎡),거래금액(만원),층,건축년도,계약날짜기준_건물연식,계약날짜,단가(만원/㎡),건물연식,부동산유형
110,서울특별시 강남구 개포동 1174-11,논현로6길 26-9,지호빌라B,32.89,24050,3,2016.0,0.0,2016-11-29,731.225296,5.0,연립다세대
111,서울특별시 강남구 개포동 1174-11,논현로6길 26-9,지호빌라B,32.89,24050,2,2016.0,0.0,2016-11-29,731.225296,5.0,연립다세대
112,서울특별시 강남구 개포동 1174-11,논현로6길 26-9,지호빌라B,33.06,22950,5,2016.0,0.0,2016-11-29,694.192377,5.0,연립다세대
113,서울특별시 강남구 개포동 1174-11,논현로6길 26-9,지호빌라B,39.91,28950,3,2016.0,0.0,2016-11-29,725.382110,5.0,연립다세대
114,서울특별시 강남구 개포동 1174-11,논현로6길 26-9,지호빌라B,39.91,28950,2,2016.0,0.0,2016-11-29,725.382110,5.0,연립다세대


In [11]:
idx_to_drop = []
for addr in tqdm(concat_selected_df['지번주소'].unique()):
    addr_df = concat_selected_df[concat_selected_df['지번주소'] == addr]
    if addr_df['건축년도'].nunique() > 1:
        non_max_df = addr_df[addr_df['건축년도'] != addr_df['건축년도'].max()]
        [idx_to_drop.append(i) for i in non_max_df.index]

100%|█████████████████████████████████████████████████████████████████████████████| 1764/1764 [00:02<00:00, 665.59it/s]


In [12]:
len(idx_to_drop)

0

In [13]:
bulktrade_basedir = './국토교통부_실거래가_공개시스템/집값분석/일괄구매/10호이상/'

In [14]:
%%time
concat_selected_df.to_csv(bulktrade_basedir+'연립다세대_오피스텔_일괄구매_전체.csv')

Wall time: 276 ms


In [15]:
concat_selected_df['소유자'] = np.nan

In [16]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager

In [17]:
driver = webdriver.Chrome(ChromeDriverManager().install())



====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [C:\Users\max6296\.wdm\drivers\chromedriver\win32\91.0.4472.101\chromedriver.exe] found in cache


In [20]:
driver.get("http://www.iros.go.kr/PMainJ.jsp")

In [21]:
driver.get("http://www.iros.go.kr/frontservlet?cmd=RISUWelcomeViewC")

In [22]:
def get_owner_name(df):
    df = df.copy()
    
    count = 0
    for addr in tqdm(df['지번주소'].unique()):
        owner_name = np.nan
        addr = addr.replace('-0', '')
        
        addr_df = df[df['지번주소'] == addr]
        driver.switch_to.default_content()
        
        
        driver.switch_to.frame("inputFrame")
        
        elem_search = driver.find_element_by_xpath('html/body/form[1]/div[24]/div/div/div/fieldset/div/table/tbody/tr[5]/td/span/input')
        elem_search.clear()
        elem_search.send_keys(addr)
        elem_search.send_keys(Keys.RETURN)
        time.sleep(2)
        
        try:
            driver.switch_to.alert.accept()
        except:
            pass
        
        driver.switch_to.default_content()
        driver.switch_to.frame('resultFrame')
        driver.switch_to.frame('frmOuterModal')
        
        try:
            driver.find_element_by_xpath('/html/body/div[2]/div[2]/table/tbody/tr[3]/td[6]/button').click()
        except:
            continue
        
        time.sleep(0.5)
        
        elem_result = driver.find_element_by_xpath('/html/body/div[2]/div[2]/table/tbody/tr/td[3]')
        
        owner_name = elem_result.text
        
        driver.switch_to.default_content()
        
        df.loc[addr_df.index, '소유자'] = owner_name
        
        count += 1
    
    return df

In [23]:
prac_df = get_owner_name(concat_selected_df)

100%|████████████████████████████████████████████████████████████████████████████| 1764/1764 [1:25:36<00:00,  2.91s/it]


In [24]:
concat_selected_df = prac_df.copy()

In [25]:
driver.close()

In [26]:
concat_selected_df.reset_index(drop=True, inplace=True)

In [27]:
concat_selected_df.to_csv(bulktrade_basedir+'연립다세대_오피스텔_일괄구매_전체_소유자_포함.csv', index=False)

In [ ]:
yunrip_selected_df.head()

In [ ]:
yunrip_selected_df['소유자'].value_counts()

In [ ]:
inv_df = yunrip_selected_df[
    yunrip_selected_df['소유자'] == '에***~'
]
print(inv_df.shape)
inv_df.head()

In [ ]:
count = 0
for addr in inv_df['지번주소'].unique():
    addr_df = inv_df[inv_df['지번주소'] == addr]
    
    print(addr_df.shape)
    display(addr_df)
    
    count += 1
    if count > 5:
        break

In [ ]:
print(yunrip_selected_df.shape)
yunrip_selected_df = yunrip_selected_df[
    (yunrip_selected_df['소유자'] == '서***~')
    |(yunrip_selected_df['소유자'] == '한***~')
    |(yunrip_selected_df['소유자'] == '에***~')
]
print(yunrip_selected_df.shape)
yunrip_selected_df.head()

In [ ]:
officetel_selected_df['소유자'].value_counts()

In [ ]:
inv_df = officetel_selected_df[
    officetel_selected_df['소유자'] == '중***~'
]
print(inv_df.shape)
inv_df.head()

In [ ]:
count = 0
for addr in inv_df['지번주소'].unique():
    addr_df = inv_df[inv_df['지번주소'] == addr]
    
    print(addr_df.shape)
    display(addr_df)
    
    count += 1
    if count > 5:
        break

In [ ]:
officetel_selected_df = officetel_selected_df[
    (officetel_selected_df['소유자'] == '서***~')
    |(officetel_selected_df['소유자'] == '한***~')
]
print(officetel_selected_df.shape)
officetel_selected_df.head()

In [ ]:
count = 0
for addr in officetel_selected_df['지번주소'].unique():
    addr_df = officetel_selected_df[officetel_selected_df['지번주소'] == addr]
    
    print(addr_df.shape)
    display(addr_df)
    
    count += 1
    if count > 5:
        break

In [ ]:
yunrip_selected_df['지번주소'].nunique()

In [ ]:
yunrip_selected_df.head()

In [ ]:
yunrip_selected_df['구'] = yunrip_selected_df['지번주소'].apply(lambda x: x.split(' ')[1])
yunrip_selected_df['동'] = yunrip_selected_df['지번주소'].apply(lambda x: x.split(' ')[2])

In [ ]:
yunrip_selected_df.head()

In [ ]:
yunrip_selected_df['구'].value_counts()

In [ ]:
def mean_prices_per_size(df):
    df = df.copy()
    
    df['크기별_평균단가'] = df[['지번주소', '건축년도', '계약날짜', '전용면적(㎡)', '단가(만원/㎡)']].groupby(['지번주소', '건축년도', '계약날짜', '전용면적(㎡)']).transform('mean')['단가(만원/㎡)']
    df['크기별_평균총액'] = df[['지번주소', '건축년도', '계약날짜', '전용면적(㎡)', '거래금액(만원)']].groupby(['지번주소', '건축년도', '계약날짜', '전용면적(㎡)']).transform('mean')['거래금액(만원)']
    df = df[['지번주소', '건축년도', '건물연식', '계약날짜', '계약날짜기준_건물연식', '전용면적(㎡)', '크기별_평균총액', '크기별_평균단가']]
    df = df.drop_duplicates().reset_index(drop=True)
    
    return df

In [ ]:
yunrip_short_df = mean_prices_per_size(yunrip_selected_df)
print(yunrip_short_df.shape)
yunrip_short_df.head()